色々な面白いトピックがありますが，今日はFibnacciをベースに色々な書き方で書いていきます。

主に
* Type Hint
* Private Method
* Magic Method(Dunder)
* Decorator
* List Comprehension/Loop/Generator



# Fibonacci Seq
今日はFibonacciの公式をベースにコードを書いていくので，念の為

```
F0 = 0
F1 = 1 
Fn = Fn-1 + Fn-2
```

## 環境設定
* pipがよく使われるLibのインストールツール
* 環境のバージョン管理とかはDev Container,Poetry, virtualenvなどを使うことがある


## Syntax 
* 細かいSyntaxは https://peps.python.org/pep-0008/ 
* Linterなど色々ツールがあるので試してみてください。私はflake8
* Naming conventionとかPrivate Method の名前の付け方はLinterでカバーしきれないのでDocを一読してみてくだい
* Guideline はhttps://google.github.io/styleguide/pyguide.html　を参考にしてます


In [1]:
!python -V

Python 3.9.13


## まずは最初に簡単なFibonacciのMethod書いてみます

In [2]:
def fib(n):
    if n <= 1:
        return n
    else:
        return fib(n-1) + fib(n-2)


### これにTypeHintを付けてみます

In [27]:
def fib(n: int) -> int: 
    if n <= 1:
        return n
    else:
        return fib(n-1) + fib(n-2)

## Classを追加してみます

In [26]:
class Fibonacci:
    def calculate(self, n: int) -> int:
        if n <= 1:
            return n
        else:
            return self.calculate(n-1) + self.calculate(n-2)

## Private Variable を追加してみます

private variable/method は_varialbe みたいな感じで名前をつけます。

In [5]:
class Fibonacci:
    def __init__(self):
        self._cache = {}

    def calculate(self, n: int) -> int:
        if n in self._cache:
            return self._cache[n]
        if n <= 1:
            return n
        else:
            self._cache[n] = self.calculate(n-1) + self.calculate(n-2)
            return self._cache[n]

# fib = Fibonacci()
# fib.calculate(5)
# fib._cache
# print(dir(Fibonacci))

{2: 1, 3: 2, 4: 3, 5: 5}

Private variable/method はName Convensionというだけなので，アクセスはできます。

## Private method を"__" に変えてみます


In [12]:
class Fibonacci:
    def __init__(self):
        self._cache = {}

    def __calculate(self, n: int) -> int:
        if n in self._cache:
            return self._cache[n]
        if n <= 1:
            return n
        else:
            self._cache[n] = self.__calculate(n-1) + self.__calculate(n-2)
            return self._cache[n]
        
# print(dir(Fibonacci))
# fib = Fibonacci()
# fib._Fibonacci__calculate(5)
# fib.__calculate(5)

## Magic Method を使ってみます


In [44]:
class Fibonacci:
    def __getitem__(self, n: int) -> int:
        if n <= 1:
            return n
        else:
            return self[n-1] + self[n-2]
# fib = Fibonacci()
# print(fib[10])

In [9]:
class AdvancedFibonacci:
    def __init__(self):
        self._cache: dict[int, int] = {}

    def _calculate(self, n: int) -> int:
        if n in self._cache:
            return self._cache[n]
        if n <= 1:
            return n
        else:
            result = self._calculate(n-1) + self._calculate(n-2)
            self._cache[n] = result
            return result

    def __getitem__(self, n: int) -> int:
        return self._calculate(n)

    def __add__(self, other: 'AdvancedFibonacci') -> int:
        # Override addition to perform multiplication
        if isinstance(other, AdvancedFibonacci):
            return self.__getitem__(10) * other.__getitem__(10)
        else:
            raise TypeError("Unsupported operand type(s) for +: 'AdvancedFibonacci' and '{}'".format(type(other).__name__))

# Example usage
fib1 = AdvancedFibonacci()
fib2 = AdvancedFibonacci()

print(fib1 + fib2) 

3025


## 複雑になってきたので，一度リセットして，Decoratorの説明をします

In [7]:
from typing import Callable

def my_decorator(func: Callable[[], None]) -> Callable[[], None]:
    def wrapper() -> None:
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

@my_decorator
def say_hello() -> None:
    print("Hello!")

say_hello()

Something is happening before the function is called.
Hello!
Something is happening after the function is called.


### DecoratorをFibに応用してみます

In [11]:
def memoize(func):
    cache = {}
    def wrapper(n):
        if n not in cache:
            cache[n] = func(n)
        return cache[n]
    return wrapper

@memoize
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

# Loop とかListを使って書いていきます

### Simple Loop

In [72]:
def multiply_two(items: list[int]) -> list[int]: 
    output = []
    for item in items:
        output.append(item*2)
    return output

### List Comprehension

In [53]:
def multiply_two_improve(items: list[int]) -> list[int]:
    return [item*2 for item in items]

[2, 4, 6]

### Generator Function (Iterative Object)

In [79]:
from typing import Iterator

def multiply_two_generator(items: list[int]) -> Iterator[int]:
    for item in items:
        yield item * 2


### では，簡単にFibでも応用してみます

In [82]:
from typing import Iterator

def fib_pair(n: int) -> Iterator[tuple[int, int]]:
    a, b = 0, 1
    for _ in range(n):
        yield a, b
        a, b = b, a + b

In [84]:
def fib_list(n: int) -> list[int]:
    result = []
    for a, _ in fib_pair(n):
        result.append(a)
    return result


In [83]:
def fib_list_comprehension(n: int) -> list[int]:
    return [a for a, _ in fib_pair(n)]